# Tiny Music Transformer (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (GPU CHECK)

In [ ]:
#@title NVIDIA GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone --depth 1 https://github.com/asigalov61/Tiny-Music-Transformer
!pip install huggingface_hub
!pip install torch
!pip install einops
!pip install torch-summary
!pip install tqdm
!pip install matplotlib
!apt install fluidsynth #Pip does not work for some reason. Only apt works

In [ ]:
#@title Import modules

print('=' * 70)
print('Loading core Tiny Music Transformer modules...')

import os
import copy
import pickle
import secrets
import statistics
from time import time
import tqdm

print('=' * 70)
print('Loading main Tiny Music Transformer modules...')
import torch

%cd /content/Tiny-Music-Transformer

import TMIDIX

from midi_to_colab_audio import midi_to_colab_audio

from x_transformer_1_23_2 import *

import random

%cd /content/
print('=' * 70)
print('Loading aux Tiny Music Transformer modules...')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

from IPython.display import Audio, display

from huggingface_hub import hf_hub_download

from google.colab import files

print('=' * 70)
print('Done!')
print('Enjoy! :)')
print('=' * 70)

# (LOAD MODEL)

In [ ]:
#@title Load Tiny Music Transformer Pre-Trained Model

#@markdown Choose model

select_model_to_load = "139M-32L-Very-Fast-Tiny" # @param ["139M-32L-Very-Fast-Tiny"]

#@markdown Model precision option

model_precision = "bfloat16" # @param ["bfloat16", "float16"]

#@markdown bfloat16 == Half precision/faster speed (if supported, otherwise the model will default to float16)

#@markdown float16 == Full precision/fast speed

plot_tokens_embeddings = "None" # @param ["None", "Start Times", "Durations Velocities", "Piano Pitches", "Violin Pitches", "Aux"]

full_path_to_models_dir = "/content/Tiny-Music-Transformer/Model/"

print('=' * 70)
print('Loading Tiny Music Transformer Training Data...')
print('Please wait...')
print('=' * 70)

training_data_file = 'Tiny_Music_Transformer_Train_Data_Mono_Melodies_Piano_Violin_MIDI_Dataset.pickle'

if os.path.isfile(full_path_to_models_dir+training_data_file):
  print('Training data already exists...')

else:
  hf_hub_download(repo_id='asigalov61/Tiny-Music-Transformer',
                  filename=training_data_file,
                  local_dir='/content/Tiny-Music-Transformer/Model',
                  local_dir_use_symlinks=False)

print('=' * 70)
train_data = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/Tiny-Music-Transformer/Model/Tiny_Music_Transformer_Train_Data_Mono_Melodies_Piano_Violin_MIDI_Dataset')
print('Done!')
print('=' * 70)

print('Loading Tiny Music Transformer', select_model_to_load,'Pre-Trained Model...')
print('Please wait...')
print('=' * 70)

if select_model_to_load == '139M-32L-Very-Fast-Tiny':

  model_checkpoint_file_name = 'Tiny_Music_Transformer_Tiny_Trained_Model_10737_steps_0.4039_loss_0.8729_acc.pth'
  model_path = full_path_to_models_dir+model_checkpoint_file_name
  if os.path.isfile(model_path):
    print('Model already exists...')

  else:
    hf_hub_download(repo_id='asigalov61/Tiny-Music-Transformer',
                    filename=model_checkpoint_file_name,
                    local_dir='/content/Tiny-Music-Transformer/Model',
                    local_dir_use_symlinks=False)



print('=' * 70)
print('Instantiating model...')

torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda'

if model_precision == 'bfloat16' and torch.cuda.is_bf16_supported():
  dtype = 'bfloat16'
else:
  dtype = 'float16'

if model_precision == 'float16':
  dtype = 'float16'

ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

SEQ_LEN = 8192

# instantiate the model

model = TransformerWrapper(
    num_tokens = 644,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 512, depth = 32, heads = 16, attn_flash = True)
)

model = AutoregressiveWrapper(model, ignore_index=643)

model.cuda()
print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(model_path))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)

print('Model will use', dtype, 'precision...')
print('=' * 70)

# Model stats
print('Model summary...')
summary(model)

# Plot Token Embeddings
if plot_tokens_embeddings != 'None':
  tok_emb = model.net.token_emb.emb.weight.detach().cpu().tolist()

if plot_tokens_embeddings == 'Start Times':
  tok_range = [0, 128]

elif plot_tokens_embeddings == 'Durations Velocities':
  tok_range = [128, 256]

elif plot_tokens_embeddings == 'Piano Pitches':
  tok_range = [256, 384]

elif plot_tokens_embeddings == 'Violin Pitches':
  tok_range = [384, 512]

elif plot_tokens_embeddings == 'Aux':
  tok_range = [512, 643]

if plot_tokens_embeddings != 'None':

  tok_emb1 = []

  for t in tok_emb[tok_range[0]:tok_range[1]]:
    tok_emb1.append(t)

  cos_sim = metrics.pairwise_distances(
    tok_emb1, metric='cosine'
  )
  plt.figure(figsize=(7, 7))
  plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
  im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
  plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
  plt.xlabel("Position")
  plt.ylabel("Position")
  plt.tight_layout()
  plt.plot()
  plt.savefig("/content/Tiny-Music-Transformer-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# (GENERATE)

# (PRIMED CONTINUATION)

In [ ]:
#@title Primed Continuation

#@markdown Generation settings

number_of_prime_tokens = 512 # @param {type:"slider", min:3, max:2048, step:3}
number_of_tokens_to_generate = 1024 # @param {type:"slider", min:30, max:8190, step:3}
number_of_batches_to_generate = 8 #@param {type:"slider", min:1, max:16, step:1}
temperature = 0.9 # @param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown Other settings
include_prime_tokens_in_generated_output = True #@param {type:"boolean"}
render_MIDI_to_audio = True # @param {type:"boolean"}

print('=' * 70)
print('Tiny Music Transformer Primed Continuation Model Generator')
print('=' * 70)

random.shuffle(train_data)

melody_chords = random.choice(train_data)

outy = melody_chords[:number_of_prime_tokens]

torch.cuda.empty_cache()

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

with ctx:
  out = model.generate(inp,
                        number_of_tokens_to_generate,
                        temperature=temperature,
                        return_prime=include_prime_tokens_in_generated_output,
                        verbose=True)

out0 = out.tolist()

torch.cuda.empty_cache()

print('=' * 70)
print('Done!')
print('=' * 70)

#======================================================================
print('Rendering results...')

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:12])
  print('=' * 70)

  if len(out) != 0:

      song = out1
      song_f = []

      time = 0
      dur = 0
      vel = 90
      pitch = 0
      channel = 0

      patches = [0] * 16
      patches[3] = 40

      for ss in song:

          if 0 <= ss < 128:

              time += (ss * 16)

          if 128 <= ss < 256:

              dur = (ss-128) * 16

          if 256 <= ss < 512:

              pitch = (ss-256) % 128

              channel = (ss-256) // 128

              if channel == 1:
                channel = 3
                vel = 110
              else:
                channel = 0
                vel = 80

              song_f.append(['note', time, dur, channel, pitch, vel, patches[channel] ])

      patches = [0 if x==-1 else x for x in patches]

      detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                                output_signature = 'Tiny Music Transformer',
                                                                output_file_name = '/content/Tiny-Music-Transformer-Music-Composition_'+str(i),
                                                                track_name='Project Los Angeles',
                                                                list_of_MIDI_patches=patches
                                                                )
      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Tiny-Music-Transformer-Music-Composition_'+str(i)

      if render_MIDI_to_audio:
        midi_audio = midi_to_colab_audio(fname + '.mid')
        display(Audio(midi_audio, rate=16000, normalize=False))

      TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

# Congrats! You did it! :)